Import packages.

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

Reuse forward Euler method from previous example:

In [2]:
def forward_euler(func: Callable, c0: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Generic forward euler method for initial value problem.

    Parameters
    ----------
    func : Callable
        ODE system to be solved
    c0 : np.ndarray
        Initial condition
    t : np.ndarray
        Time grid points

    Returns
    -------
    np.ndarray
        Solution of ODE system
    """
    # initialize arrays for time and solution values
    c = np.zeros((len(c0),len(t)))
    h = t[1]-t[0]

    # initial condition
    c[:,0] = c0

    # iterate over each time step
    for i in range(1,len(t)):
        c[:,i] = c[:,i-1]+h*func(t[i-1], c[:,i-1])
    return c

Reuse backward Euler method from previous example:

In [3]:
def backward_euler(func: Callable, c0: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Generic backward Euler method for initial value problem. Use scipy's 
    fsolve to solve root finding problem.

    Parameters
    ----------
    func : Callable
        Function that defines the ODE (y' = func(t, y)).
    c0 : np.ndarray
        Initial condition.
    t : np.ndarray
        Time domain.

    Returns
    -------
    np.ndarray
        Array of solution values at the time points.
    """
    # initialize arrays for time and solution values
    c = np.zeros([len(c0),len(t)])
    dt = t[1]-t[0]

    # initial condition
    c[:,0] = c0
    
    # iterate over each time step
    for i in range(len(t)-1):
        
        # initial guess for y_{i+1}
        c_guess = c[:,i]

        # define backward Euler function
        euler = lambda c_next: c[:,i] + dt * func(t[i+1], c_next) - c_next

        # update solution
        c[:,i+1] = scipy.optimize.fsolve(euler, c_guess)
    
    return c

## Stability

We have the following example

$A {\overset{k}{\rightarrow}} B$

Initial conditions:

$C_A(0)=1\, mol/L$

$C_B(0)=0\, mol/L$

Rate constant:

$k=0.2\, min^{-1}$

ODE for $C_A$ :

$\frac{dC_A}{dt} = -kC_A $

The concentration of B follows from the following algebraic relationship assuming constant total concentration equal to 1: 

$ C_B = 1 - C_A $

Analytical solutions:

$ C_A(t)=e^{-kt} $

$ C_B(t)=1 - e^{-kt} $

Define parameters

In [4]:
# ODE for CA
def dcA(t, c):
    return -0.2*c

# initial condition
c0 = np.array([1])

# analytical solution of ODE
def cA_analytical(k: float, t: float) -> float:
    return np.exp(-k*t)

Define a fine and a coarse time grid.

In [5]:
t_fine = np.linspace(0,50,51)
h_fine = (t_fine[-1]-t_fine[0])/(len(t_fine)-1)
c_fine = forward_euler(dcA, c0, t_fine)

t_coarse = np.linspace(0,50,5)
h_coarse = (t_coarse[-1]-t_coarse[0])/(len(t_coarse)-1)
c_coarse_forward = forward_euler(dcA, c0, t_coarse)
c_coarse_backward = backward_euler(dcA, c0, t_coarse)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_fine, c_fine[0,:], label = f"forward Euler, h={h_fine}")
ax.plot(t_coarse, c_coarse_forward[0,:], label = f"forward Euler, h={h_coarse}")
ax.plot(t_coarse, c_coarse_backward[0,:], label = f"backward Euler, h={h_coarse}")
ax.plot(t_fine, cA_analytical(0.2, t_fine), label = "analytical solution", linestyle = "dashed")
ax.set_xlabel("time [s]")
ax.set_ylabel("concentration [mol/L]")
ax.set_title("Stability Analysis")
ax.legend()
ax.grid()
fig.show()